In [11]:
import pandas as pd
import re

Load Kenpom

In [107]:
pd.set_option('display.max_columns', 25)
df_html = pd.read_html('https://kenpom.com/index.php?y=2019')

In [108]:
df = df_html[0]
df.columns = df.columns.get_level_values(1)

Get league average Tempo

In [109]:
df_small = df.iloc[:,[1,2,4,5,7,9,11,15,17]].copy()

In [110]:
#Remove line break rows
df_small_fix = df_small.drop(
    [
        df_small.index[40],
        df_small.index[41],
        df_small.index[82],
        df_small.index[83],
        df_small.index[124],
        df_small.index[125],
        df_small.index[166],
        df_small.index[167],
        df_small.index[208],
        df_small.index[209],
        df_small.index[250],
        df_small.index[251],
        df_small.index[292],
        df_small.index[293],
        df_small.index[334],
        df_small.index[335],
    ]
)

In [111]:
df_small_fix['AdjT'] = pd.to_numeric(df_small_fix['AdjT'])
df_small_fix['AdjO'] = pd.to_numeric(df_small_fix['AdjO'])
df_small_fix['AdjD'] = pd.to_numeric(df_small_fix['AdjD'])
df_small_fix['AdjEM'] = pd.to_numeric(df_small_fix['AdjEM'])
df_small_fix['Luck'] = pd.to_numeric(df_small_fix['Luck'])
df_small_fix['OppO'] = pd.to_numeric(df_small_fix['OppO'])
df_small_fix['OppD'] = pd.to_numeric(df_small_fix['OppD'])

In [112]:
laT = df_small_fix['AdjT'].mean()
laT

67.76770538243626

Get league average points per posession

In [113]:
Oppp = df_small_fix['AdjO'].mean()
Dppp = df_small_fix['AdjD'].mean()

GAMEDAY CALCULATION

In [126]:
team_list = list(df_small_fix['Team'])
new_team_list = []
for team in team_list:
    f = re.split(r'\s+(?=\d)|(?<=\d)\s+', team)[0]
    new_team_list.append(f)

df_small_fix['Team'] = new_team_list

In [187]:
away_team = "Saint Louis"
home_team = "Virginia Tech"

df_small_fix[df_small_fix['Team'].isin([home_team, away_team])]

,Team,Conf,AdjEM,AdjO,AdjD,AdjT,Luck,OppO,OppD
12,Virginia Tech,ACC,24.07,118.1,94.0,63.7,-0.005,108.3,99.9
109,Saint Louis,A10,6.02,102.2,96.2,66.4,0.044,104.4,103.9


In [188]:
# Get HOME TEAM FIXTURES
hadjO = df_small_fix[df_small_fix['Team'] == home_team]['AdjO'].values[0]
hadjD = df_small_fix[df_small_fix['Team'] == home_team]['AdjD'].values[0]
hadjT = df_small_fix[df_small_fix['Team'] == home_team]['AdjT'].values[0]
hluck = df_small_fix[df_small_fix['Team'] == home_team]['Luck'].values[0]
hOppO = df_small_fix[df_small_fix['Team'] == home_team]['OppO'].values[0]
hOppD = df_small_fix[df_small_fix['Team'] == home_team]['OppD'].values[0]

# Get AWAY TEAM Fixtures
aadjO = df_small_fix[df_small_fix['Team'] == away_team]['AdjO'].values[0]
aadjD = df_small_fix[df_small_fix['Team'] == away_team]['AdjD'].values[0]
aadjT = df_small_fix[df_small_fix['Team'] == away_team]['AdjT'].values[0]
aluck = df_small_fix[df_small_fix['Team'] == away_team]['Luck'].values[0]
aOppO = df_small_fix[df_small_fix['Team'] == away_team]['OppO'].values[0]
aOppD = df_small_fix[df_small_fix['Team'] == away_team]['OppD'].values[0]

In [189]:
#Predicted pace
xp = ( (hadjT * aadjT) / laT )
xp

62.41439010116211

In [190]:
# adjusted effiency differential logic
#if (hadjO > 100):
#    if (aadjD > 100):
#        if (hOppD < 100) or ((aadjD - hOppD) > 0):
#            hadjO = hadjO + (aadjD - hOppD)
            #if (hadjT > 72):
                #hadjO = (hadjO * (hadjT / laT))
        
## adjusted efficiency differential logic
if (hadjO > 114):
    #diminishing returns
    if (hadjO - aadjD) > 10 & (aadjT < 68):
        if (hadjT < 63):
            hadjO = hadjO - 6.26
        elif (hadjT > 72):
            hadjO = hadjO - 7.94
        else:
            hadjO = hadjO - 5.88
    if (hadjO - aadjD) > 10 & (aadjT > 70.5):
        if (hadjT > 69):
            hadjO = hadjO - 1.62

# adjusted home offensive efficiency vs quality of away team offenses
lgadjOh = ((hadjO / aOppO) * Dppp)
print(lgadjOh)
#xhps = hps * lgadjOa

# away points allowed
apa = aadjD + (aadjD *(.000))

# multiply expected home score by expected away points allowed over league average points per possession
hproj = (lgadjOh * apa) / Oppp
hprojT = round(((hproj * xp) / 100),0)

#print("home team points per possession", hproj, "home team points by predicted tempo", xp, hprojT)
#print(home_team, hprojT)

112.12956188336427


In [191]:
# adjusted home defensive efficiency differential logic

# Good defense + fast team
if (hadjD < 95) 

# Good defense + slow team
if 

# Slog

# good defense average pace

    
# home points allowed
hpa = hadjD
    
# adjusted away offensive efficiency differential logic

# Slow as balls and bad offense

# Slow as balls average offense

# slow as balls good offense

# fast and good offense


# Away projected points scored (MORG) adjusted to league average points per possession
lgadjOa = ((aadjO / hOppO) * Dppp)
print(lgadjOa)
#xaps = aps * lgadjOh

# multiply expected home score by expected away points allowed over league average points per possession
aproj = (lgadjOa * hpa) / Oppp
aprojT = round(((aproj * xp) / 100), 0)

#print("away team points per possession", aproj, "away team points by predicted tempo", xp, aprojT)
#print(away_team, aprojT)

84.64707414876837


In [192]:
#home spread
spread = hprojT - aprojT
#projected total
total = hprojT + aprojT

print(home_team, hprojT)
print(away_team, aprojT)
print("Total: ", total)

Virginia Tech 65.0
Saint Louis 46.0
Total:  111.0
